In [1]:
dates_list = ['15/02/2022','25/01/2022','14/01/2022','18/12/2021','14/12/2021','26/11/2021','12/11/2021','02/11/2021','14/09/2021','13/08/2021','09/07/2021','18/06/2021','28/05/2021','11/05/2021','20/04/2021','13/04/2021','23/03/2021','09/03/2021','23/02/2021','02/02/2021','20/01/2021','12/01/2021','14/12/2020','17/11/2020','03/11/2020','13/10/2020','28/09/2020','25/09/2020','01/09/2020','18/08/2020','06/08/2020','24/06/2020','19/05/2020','06/05/2020','21/04/2020','31/03/2020','23/03/2020','15/03/2020','12/03/2020']

dates_list

['15/02/2022',
 '25/01/2022',
 '14/01/2022',
 '18/12/2021',
 '14/12/2021',
 '26/11/2021',
 '12/11/2021',
 '02/11/2021',
 '14/09/2021',
 '13/08/2021',
 '09/07/2021',
 '18/06/2021',
 '28/05/2021',
 '11/05/2021',
 '20/04/2021',
 '13/04/2021',
 '23/03/2021',
 '09/03/2021',
 '23/02/2021',
 '02/02/2021',
 '20/01/2021',
 '12/01/2021',
 '14/12/2020',
 '17/11/2020',
 '03/11/2020',
 '13/10/2020',
 '28/09/2020',
 '25/09/2020',
 '01/09/2020',
 '18/08/2020',
 '06/08/2020',
 '24/06/2020',
 '19/05/2020',
 '06/05/2020',
 '21/04/2020',
 '31/03/2020',
 '23/03/2020',
 '15/03/2020',
 '12/03/2020']